## LG화학 학습데이터 생성  
lg_chem 폴더 내에 db.sqlite, 불용어 파일, 태그 파일(tag.xlsx로 파일명 변환), dx_tag 폴더를 넣은 상태에서 돌리면 결과 재현 가능

In [1]:
import sqlite3

import os
from time import time
import re
import numpy as np
import pandas as pd

from nltk.tokenize import sent_tokenize
from ckonlpy.tag import Twitter
twitter = Twitter()
from konlpy.tag import Okt
okt = Okt()
# from pororo import Pororo
# prr = Pororo(task="tokenization", lang="ko", model="mecab_ko")
# prr = Pororo(task="tokenization", lang="ko", model="sent_ko")

import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\jinho.lee\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


### A. 태그 사전 생성

In [2]:
db_path        = './'
dx_path        = './dx_tag/'
data_path      = "./chem_data/"
tag_data_path  = "./tag_data/"
stopwords_path = './'

In [4]:
tag_data_df = pd.read_excel('./tag.xlsx')
tag_data_df = tag_data_df.drop_duplicates(['키워드'], keep='first').reset_index(drop=True)

In [5]:
label_lst = tag_data_df['TAG'][-tag_data_df['TAG'].isna()].unique().tolist()
# ['기관', '기술', '소재', '업체', '이벤트', '이슈', '정책', '제품']

global label_dict # 함수 내에서 사용. LG화학의 경우 태그가 한글로 되어있어서 변경하고자 함수를 수정하였습니다.
# 'LABEL-C'은 화학 고유의 태그이고, DX tag와 merge된 '-C'가 빠진 파일명을 가질 예정입니다.
label_dict = {'기관' : 'ORG-C', '기술' : 'TE-C', '소재' : 'MAT', '업체' : 'COM-C', '이벤트' : 'EVT-C', '이슈' : 'ISS', '정책' : 'POL-C', '제품' : 'PRD'}

In [6]:
def make_ref_dic(label_lst, dic_data_df):
    total_dic_cnt = len(dic_data_df[dic_data_df['TAG']!=None])
    print('Total words: {0:>5,}'.format(total_dic_cnt))
    for label in label_lst:
        try: 
            dic_lst = dic_data_df[dic_data_df['TAG']==label]['키워드'].tolist()
            dic_lst = list(set(dic_lst))
            save_dic_df = pd.DataFrame(dic_lst, columns=[label_dict[label]])

            # Create path if it does not exist
            if not os.path.exists(tag_data_path):
                os.makedirs(tag_data_path, exist_ok=True)
                print("{} -- Folder create complete \n".format(tag_data_path))

            save_dic_df.to_excel(tag_data_path+label_dict[label]+".xlsx")
            print('tag : {} | cnt : {}'.format(label_dict[label], len(dic_lst)))
        except:
            print("error occurred while processing label " + label_dict[label])

In [7]:
make_ref_dic(label_lst, tag_data_df)

Total words: 117,692
./tag_data/ -- Folder create complete 

tag : ORG-C | cnt : 2248
tag : TE-C | cnt : 1016
tag : MAT | cnt : 951
tag : COM-C | cnt : 2785
tag : EVT-C | cnt : 3448
tag : ISS | cnt : 444
tag : POL-C | cnt : 608
tag : PRD | cnt : 845


### B. Merge DX tag -> Chem tag  
\- BM, LOC tag들은 파일 탐색창에서 manually 복붙하였습니다.

In [8]:
# CO (dx) + COM-C (chem) -> COM
com = set(pd.read_excel(tag_data_path+'COM-C.xlsx')['COM-C'].tolist())
co = set(pd.read_excel(dx_path+'CO.xlsx')['CO'].tolist())
com = co.union(com)
pd.DataFrame(list(com), columns=['COM']).to_excel(tag_data_path+'COM.xlsx')

In [9]:
# EVT (dx) + EVT-C (chem) -> EVT
evt = set(pd.read_excel(tag_data_path+'EVT-C.xlsx')['EVT-C'].tolist())
evt_dx = set(pd.read_excel(dx_path+'EVT.xlsx')['EVT'].tolist())
evt = evt.union(evt_dx)
pd.DataFrame(list(evt), columns=['EVT']).to_excel(tag_data_path+'EVT.xlsx')

In [10]:
# ORG (dx) + ORG-C (chem) -> ORG
org = set(pd.read_excel(tag_data_path+'ORG-C.xlsx')['ORG-C'].tolist())
org_dx = set(pd.read_excel(dx_path+'ORG.xlsx')['ORG'].tolist())
org = org.union(org_dx)
pd.DataFrame(list(org), columns=['ORG']).to_excel(tag_data_path+'ORG.xlsx')

In [11]:
# PO (dx) + POL-C (chem) -> POL
pol = set(pd.read_excel(tag_data_path+'POL-C.xlsx')['POL-C'].tolist())
po_dx = set(pd.read_excel(dx_path+'PO.xlsx')['PO'].tolist())
pol = pol.union(po_dx)
pd.DataFrame(list(pol), columns=['POL']).to_excel(tag_data_path+'POL.xlsx')

In [12]:
# TE (dx) + TE-C (chem) -> TE
te = set(pd.read_excel(tag_data_path+'TE-C.xlsx')['TE-C'].tolist())
te_dx = set(pd.read_excel(dx_path+'TE.xlsx')['TE'].tolist())
te = te.union(te_dx)
pd.DataFrame(list(te), columns=['TE']).to_excel(tag_data_path+'TE.xlsx')

In [13]:
tag_label_lst = ['BM', 'COM', 'EVT', 'ISS', 'LOC', 'MAT', 'ORG', 'POL', 'PRD', 'TE']

### C. Generate train/test data

#### 1) Load article from DB

In [14]:
conn = sqlite3.connect(db_path + 'db.sqlite')
c = conn.cursor()

c.execute('''SElECT date, source, title, summary, org, link FROM article_kr_dx''')

init_df = pd.DataFrame(c.fetchall(),
                       columns=['date', 'source', 'title', 'summary', 'org', 'link'])
filtered_df = init_df.drop_duplicates(['date', 'title'], keep='first')  
sorted_df = filtered_df.sort_values(by='date', ascending=False)

stop_word_df = pd.read_excel(stopwords_path+"no_use_words_kr.xlsx")
stop_word_list = stop_word_df['word'].tolist()

conn.close()
filtered_df.head()

,date,source,title,summary,org,link
0,20201231,서울경제,골든타임도 지나고...명민호 선원 시신 발견,제주 해상에서 전복된 저인망 어선 ‘32명민호’의 선원 1명이 사망한 채 발견됐다....,viewer 지난 30일 제주해경이 제주 해상에서 전복된 명민호의 선원으로 추정되는...,http://www.sedaily.com/NewsView/1ZBWCQVY1P
1,20201231,서울경제,"[기업공시 12월 31일]ABL바이오, 면역항암제 美 FDA 1상 계획 신청 등",<유가 증권> \n \n▲아모레G(002790)=배동현 대표이사 사임 ▲대우건설(0...,"< 저작권자 ⓒ 서울경제, 무단 전재 및 재배포 금지 >\n\n=배동현 대표이사 사...",http://www.sedaily.com/NewsView/1ZBWCJX1OE
2,20201231,동아일보,[단독]은퇴 약속 지킨 서정진 “원격진료 스타트업 맨땅서 시작”,서정진 셀트리온 회장(63)이 31일 회장직에서 물러났다. ‘다른 임원과 마찬가지로...,동아일보 DB.\n\n서정진 셀트리온 회장(63)이 31일 회장직에서 물러났다. ‘...,https://www.donga.com/news/article/all/2020123...
3,20201231,서울경제,마지막까지 코로나 치료제 공들이고 떠나는 서정진 회장,서정진 셀트리온(068270) 그룹 회장이 31일 회장직에서 물러났다. \n \n지...,"viewer 서정진 셀트리온그룹 회장./권욱기자\n\n< 저작권자 ⓒ 서울경제, 무...",http://www.sedaily.com/NewsView/1ZBWCRA58P
4,20201231,서울경제,지자체 '코로나 극복 경제살리기' 성공플랜 짠다,전대미문의 신종 코로나바이러스 감염증(코로나19) 대유행이 전국을 강타하면서 202...,viewer 신축년(辛丑年)의 태양이 강원도 강릉시 사천진해변의 수평선 너머로 힘차...,http://www.sedaily.com/NewsView/1ZBWCBVQBL


In [15]:
article_df = sorted_df.drop_duplicates(['org'], keep='first').reset_index(drop=True)
article_df['org'] = article_df['org'].astype(str)
article_df.isnull().sum()

date       0
source     0
title      0
summary    0
org        0
link       0
dtype: int64

#### 2) trim too short/long articles

In [16]:
# filter by length -> 문장 개수 10000개로 맞추기
print(len(article_df))
print(len(article_df[article_df['org'].map(len) < 200]))
print(len(article_df[article_df['org'].map(len) > 1600]))
# plt.hist(article_df['org'].map(len))

fil_article_df = article_df[(article_df['org'].map(len) > 200) & (article_df['org'].map(len) < 1600)]
fil_article_df = fil_article_df.drop_duplicates().reset_index(drop=True)
len(fil_article_df)

17994
77
7012


10899

In [17]:
ar_lst = list(set(fil_article_df['org'].tolist()))
ar_lst = list(filter(lambda v: v, ar_lst)) # 결측제거
len(ar_lst)

10899

#### 3) split articles into sentences and preprocess

In [18]:
def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '\n'         # \n 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[-=+,#/\?:$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》·“”’’ⓒ━]'
    # pattern = '·“‘’[-=+,#/\\?:$@*※~&%ㆍ!』\‘'"|\(\)\[\]\<\>`\'…》]'
    text = re.sub(pattern=pattern, repl='', string=text)

    # text = text.replace("다.", "다.^")
    # text = text.replace("다 .", "다.^")
    # text = text.replace(".", "_^")
    text = text.replace("▲",'')
    text = text.replace("△",'')
    text = text.replace("■",'')
    text = text.replace("◆",'')
    text = text.replace("▶",'')
    text = text.replace("●",'')
    text = text.replace("회장",'')
    text = text.replace("회장님",'')
    text = text.replace("하현회",'')
    text = text.replace("구광모",'')
    text = text.replace("권영수",'')
    
    # pattern = '[^\w\s]'         # 특수기호제거
    # text = re.sub(pattern=pattern, repl='', string=text)
    pattern = "[A-Za-z0-9]"     # 영문/숫자제거
    # pattern = "[a-z]"     # 영문/숫자제거
    text = re.sub(pattern=pattern, repl='', string=text)
    
    return text   

In [19]:
sen_sum_lst=[]  ## sentence list
ckpt1 = time()
for i, ar in enumerate(ar_lst):  ## 각 기사에 대해서
    if i%1000==0:
        print(i, "/", len(ar_lst))
        print(time()-ckpt1, "sec")    
    split_ar = sent_tokenize(ar)  ## 문장단위로 split
    for temp_ar in split_ar:  ## 각 문장별로
        temp_ar = clean_str(temp_ar)
        temp_ar = temp_ar.strip()  ## 앞뒤 whitespace 제거
        sen_sum_lst.append(temp_ar)

0 / 10899
0.0005917549133300781 sec
1000 / 10899
0.8080470561981201 sec
2000 / 10899
1.6504991054534912 sec
3000 / 10899
2.4685349464416504 sec
4000 / 10899
3.2359139919281006 sec
5000 / 10899
4.048757076263428 sec
6000 / 10899
4.7641987800598145 sec
7000 / 10899
5.5324931144714355 sec
8000 / 10899
6.29544997215271 sec
9000 / 10899
7.044921875 sec
10000 / 10899
7.782399892807007 sec


In [20]:
sen_sum_df = pd.DataFrame(sen_sum_lst, columns=['sentens'])
len(sen_sum_df)

104925

In [21]:
fin_sen_sum_df=sen_sum_df[sen_sum_df['sentens'].map(len) > 15]
fin_sen_sum_df=fin_sen_sum_df[fin_sen_sum_df['sentens'].map(len) < 800]
print(len(fin_sen_sum_df))
fin_sen_sum_df=fin_sen_sum_df.drop_duplicates(['sentens'], keep='first')
print(len(fin_sen_sum_df))
# fin_sen_sum_lst=fin_sen_sum_df.sentens.tolist()
# fin_sen_sum_df.to_excel(tag_data_path+'fin_sen_tst.xlsx')

100622
94294


#### 4) tag and generate data

In [30]:
tag_word_sum_lst = []
for label in tag_label_lst:
    read_df = pd.read_excel(tag_data_path+label+'.xlsx')[label]
    read_lst = read_df.tolist()
    # str 타입이 아닌 값 예외처리
    for i in read_lst:
        if type(i) is not str:
            print('value not str type : ' + str(i) + ' from ' + label)
            read_lst.remove(i)
    tag_word_sum_lst.append(read_lst)
    print(label+' cnt : '+str(len(read_lst)))
    twitter.add_dictionary(read_lst, 'Noun')

# 탐색 효율성을 위해 set structure로 변환
BM_lst= set(pd.read_excel(tag_data_path+'BM.xlsx')['BM'].tolist())
COM_lst= set(pd.read_excel(tag_data_path+'COM.xlsx')['COM'].tolist())
ISS_lst= set(pd.read_excel(tag_data_path+'ISS.xlsx')['ISS'].tolist())
MAT_lst= set(pd.read_excel(tag_data_path+'MAT.xlsx')['MAT'].tolist())
EVT_lst= set(pd.read_excel(tag_data_path+'EVT.xlsx')['EVT'].tolist())
LOC_lst= set(pd.read_excel(tag_data_path+'LOC.xlsx')['LOC'].tolist())
POL_lst= set(pd.read_excel(tag_data_path+'POL.xlsx')['POL'].tolist())
PRD_lst= set(pd.read_excel(tag_data_path+'PRD.xlsx')['PRD'].tolist())
ORG_lst= set(pd.read_excel(tag_data_path+'ORG.xlsx')['ORG'].tolist())
TE_lst= set(pd.read_excel(tag_data_path+'TE.xlsx')['TE'].tolist())

BM cnt : 637
value not str type : 6528870487 from COM
COM cnt : 43284
EVT cnt : 3568
ISS cnt : 444
LOC cnt : 450
MAT cnt : 951
ORG cnt : 2393
POL cnt : 965
PRD cnt : 845
TE cnt : 1862


In [31]:
def tag(word, pos, PRD_lst, MAT_lst, TE_lst, COM_lst, ISS_lst, EVT_lst, POL_lst, BM_lst, ORG_lst, LOC_lst):
    if pos=='Noun' or pos == 'Number':
        if word in PRD_lst:
            return 'PRD-B'
        elif word in MAT_lst:
            return 'MAT-B'
        elif word in TE_lst:
            return 'TE-B'
        elif word in COM_lst:
            return 'COM-B'
        elif word in ISS_lst:
            return 'ISS-B'
        elif word in EVT_lst:
            return 'EVT-B'
        elif word in POL_lst:
            return 'POL-B'
        elif word in BM_lst:
            return 'BM-B'
        elif word in ORG_lst:
            return 'ORG-B'
        elif word in LOC_lst:
            return 'LOC-B'
        elif pos == 'Number':
            return 'NUM-B'
#         elif word == "_":
#             return "_"
        else:
            return 'O'
    else:
        return 'O'

In [32]:
input_sen=fin_sen_sum_df['sentens'][fin_sen_sum_df['sentens'].apply(lambda x: len(x)) <= 256].tolist() # BERT 계산속도 고려
print(len(input_sen))

89728


In [34]:
tagging_result_df=pd.DataFrame()
tagging_result_df=pd.DataFrame(input_sen, columns=['sentens'])
tagging_label_lst=[]
tagging_sen_lst=[]

ckpt2 = time()
for i in range(len(input_sen)): 
    if i%10000==0:
        print(i, "/", len(input_sen))
        print(time()-ckpt2, "sec")
    temp_df=pd.DataFrame(twitter.pos(input_sen[i]), columns=['word','pos'])
    temp_df["tag"] = temp_df.apply(lambda x : tag(x["word"], x["pos"], PRD_lst, MAT_lst, TE_lst, COM_lst, ISS_lst, EVT_lst, POL_lst, BM_lst, ORG_lst, LOC_lst) , axis = 1 )
    # temp_df.reset_index(drop=True, inplace=True)
    # print(temp_df)
    temp_label_lst=temp_df['tag'].tolist()
    temp_label_lst=' '.join(temp_label_lst)
    tagging_label_lst.append(temp_label_lst)
    
    temp_word_lst=temp_df['word'].tolist()
    temp_word_lst=' '.join(temp_word_lst)
    tagging_sen_lst.append(temp_word_lst)
    
tagging_result_df['label']=pd.DataFrame(tagging_label_lst)
tagging_result_df['sentens']=pd.DataFrame(tagging_sen_lst)

0 / 89728
0.0011289119720458984 sec
10000 / 89728
77.66743397712708 sec
20000 / 89728
151.76407599449158 sec
30000 / 89728
227.3166539669037 sec
40000 / 89728
308.53064489364624 sec
50000 / 89728
381.66545605659485 sec
60000 / 89728
487.33572793006897 sec
70000 / 89728
584.0805869102478 sec
80000 / 89728
662.1295757293701 sec


In [35]:
shuffled_result_df=tagging_result_df.sample(frac=1).reset_index(drop=True)
print('전체 데이터 개수',len(shuffled_result_df))
shuffled_result_df.head()

전체 데이터 개수 89728


,sentens,label
0,하 지만 행정명령 서명 으로 미국 에서 생산 되는 백신 이 해외 에 수출 되는 시간...,O O EVT-B O O LOC-B O O O COM-B O O O O O O O ...
1,한국 도 상황 에 맞는 실리콘밸리 조성 에 나섰다 .,O O O O O COM-B COM-B O O O
2,지역 의 대표 거점 산업 단지 와 인근 의 여러 산단 을 묶어 산업 혁신 공간 으로...,O O O O O O O O O O O O O O COM-B O O O O O O ...
3,📁 관련 통계 자료 다운로드 공모 주 평균 주가 상승률 공모 가 격 이 희망 가격 ...,O O O O O O O O O EVT-B O O O O O O O O O O O ...
4,각 부처 는 내년 예산 요구 의 가이드라인 으로 이를 활용 하게 된다 .,O O O O O O O O O O O O O O


In [36]:
# Create path if it does not exist
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)
    print("{} -- Folder create complete \n".format(data_path))

train_set = shuffled_result_df.sample(frac=0.8, random_state=2021)
print('train_set 개수', len(train_set))
train_set.to_excel(data_path+'train_chem.xlsx')

test_set = shuffled_result_df.drop(train_set.index)
print('test_set 개수', len(test_set))
test_set.to_excel(data_path+'test_chem.xlsx')

./chem_data/ -- Folder create complete 

train_set 개수 71782
test_set 개수 17946
